# Summary

This notebook demonstrates the use of `snscrape` social-media scrapper to efficiently (with parallelization) retrieve tweets from multiple accounts.

As an example, we will retrieve recent tweets of well-known futurists. 

In [2]:
# install social media scrapper: !pip3 install snscrape
import snscrape.modules.twitter as sntwitter
import itertools
import multiprocessing.dummy as mp # for multithreading 
import datetime
import pandas as pd

start_date = datetime.datetime(2023,2,14,tzinfo=datetime.timezone.utc) # from when
attributes = ('date','url','rawContent') # what attributes to keep

def get_tweets(username,n_tweets = 100,attributes=attributes):
    tweets = itertools.islice(sntwitter.TwitterSearchScraper(f'from:{username}').get_items(),n_tweets) # invoke the scrapper
    tweets = filter(lambda t:t.date>=start_date, tweets)
    tweets = map(lambda t: (username,)+tuple(getattr(t,a) for a in attributes),tweets) # keep only attributes needed
    tweets = list(tweets) # the result has to be pickle'able
    return tweets

# a list of accounts to scrape
user_names = ['kevin2kelly','briansolis','PeterDiamandis','Richard_Florida']

# parallelise queries for speed ! 
with mp.Pool(4) as p:
    results = p.map(get_tweets, user_names)
    # combine
    results = list(itertools.chain(*results))

In [4]:
import pandas as pd
pd.set_option('max_colwidth', 800)
pd.DataFrame(data=results,columns=('user',)+attributes)

,user,date,url,rawContent
0,kevin2kelly,2023-02-14 23:48:46+00:00,https://twitter.com/kevin2kelly/status/1625643242921562113,@jamescham So you pay but don't have a blue badge?
1,kevin2kelly,2023-02-14 22:53:15+00:00,https://twitter.com/kevin2kelly/status/1625629268993789953,"@jamescham Wait, how many characters did you just post?"
2,kevin2kelly,2023-02-14 19:01:32+00:00,https://twitter.com/kevin2kelly/status/1625570955589214232,#AI_ART #DailyAI #ImadeAIart https://t.co/Hbe3yLL33R
3,kevin2kelly,2023-02-14 16:50:58+00:00,https://twitter.com/kevin2kelly/status/1625538099148357633,"@craigmod That's ""research""."
4,kevin2kelly,2023-02-14 16:50:42+00:00,https://twitter.com/kevin2kelly/status/1625538033666904064,@rajeshbhusal @craigmod The new Bing hasn't given me useful information yet. What are you finding?
5,briansolis,2023-02-15 01:21:25+00:00,https://twitter.com/briansolis/status/1625666557756993537,"@mbkumar @HitachiVantara Thank you, Monica! 🙏"
6,briansolis,2023-02-14 16:45:59+00:00,https://twitter.com/briansolis/status/1625536843528949760,"My @Forbes story was just selected as Editor's Pick!\n\n""Learn To Speak Your Customers' Love Language Through Digital Empathy""\n\nhttps://t.co/dAxlw0tmrd https://t.co/HDehMZ2BnZ"
7,PeterDiamandis,2023-02-15 16:20:26+00:00,https://twitter.com/PeterDiamandis/status/1625892800918171651,You can learn more about the membership and this years Summit here: https://t.co/CKMuv6TVG8
8,PeterDiamandis,2023-02-15 15:55:21+00:00,https://twitter.com/PeterDiamandis/status/1625886491791482881,"Every year I host a Summit for the members of my A360 community.\n\nIt's designed for founders, executives, and investors who want to create meaningful impact, leave a legacy, and learn about the tech shaping the next decade.\n\nConsider applying if this is you. There is 1 week left"
9,PeterDiamandis,2023-02-15 14:57:16+00:00,https://twitter.com/PeterDiamandis/status/1625871873811832832,"Great Entrepreneurs create amazing User Interfaces Moments that change the world. \n\nHere's 3 top ""User Interface Moments"" over time:\n&gt;1993: Mosaic interface to ARPA-net\n&gt;2007: iPhone interface to digital world\n&gt;2022: ChatGPT interface to AI\n\nWhat other User Interface Moments?"
